In [1]:
import sys
import os

!git clone https://github.com/komalniraula/adaptive-inference-llm

repo_name = 'adaptive-inference-llm' # Must match the folder created by git clone
project_path = os.path.join('/content', repo_name)

# Append the project root directory to the system path

sys.path.append(project_path)

Cloning into 'adaptive-inference-llm'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 258 (delta 124), reused 197 (delta 65), pack-reused 0 (from 0)
Receiving objects: 100% (258/258), 6.12 MiB | 19.51 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [2]:
import pandas as pd
import numpy as np

In [3]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, ModernBertForSequenceClassification

import torch.nn.functional as F

In [11]:
# Load dataset
import time

from evaluation.dataset_loaders.sst2 import load_sst2

dataset_loaders = [
    ("sst2", load_sst2, "classification"),
    ]

cached_datasets = {}
print("Loading datasets once...\n")

for name, loader, task in dataset_loaders:
    print(f"Loading {name}...")
    cached_datasets[name] = {
        "data": loader(task='test', fraction=1),
        "task": task
    }

for data in cached_datasets:
    print(f"Length of the {data} dataset: {len(cached_datasets[data]['data'])}")
    
print("\nAll datasets loaded.\n")

Loading datasets once...

Loading sst2...
Length of the sst2 dataset: 872

All datasets loaded.



In [5]:
from transformers import AutoModelForSequenceClassification


class RoBERTaLargeBaselineClassifier:
    def __init__(self, model_name="siebert/sentiment-roberta-large-english"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load tokenizer + fine-tuned RoBERTa-large sentiment classifier
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)

        self.model.to(self.device)
        self.model.eval()

        # RoBERTa-large has 24 encoder layers
        self.num_layers = self.model.config.num_hidden_layers  # = 24

    @torch.no_grad()
    def classify(self, text, dataset_name=None):

        # Tokenize
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=False,
        )
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # Forward pass through full model
        outputs = self.model(**inputs)
        logits = outputs.logits  # shape [1, 2]

        # Softmax probabilities
        probs = F.softmax(logits, dim=-1)[0]
        conf, pred = torch.max(probs, dim=0)

        # Return: predicted label, #layers used, confidence
        return pred.item(), self.num_layers, conf.item()


In [6]:
class DistilBERTBaselineClassifier:
    def __init__(self, model_name="distilbert-base-uncased-finetuned-sst-2-english"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load tokenizer + fine-tuned DistilBERT sentiment classifier
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)

        self.model.to(self.device)
        self.model.eval()

        # DistilBERT has 6 encoder layers
        self.num_layers = self.model.config.num_hidden_layers  # = 6

    @torch.no_grad()
    def classify(self, text, dataset_name=None):
        # Tokenize text
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=False
        )
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # Forward pass
        outputs = self.model(**inputs)
        logits = outputs.logits  # shape [1, 2]

        # Compute probabilities
        probs = F.softmax(logits, dim=-1)[0]
        conf, pred = torch.max(probs, dim=0)

        # Baseline → always uses full 6 layers
        return pred.item(), self.num_layers, conf.item()

In [8]:
# Extract (text, label) from any format
def extract_text_label(sample):
    if isinstance(sample, dict):
        if "text" in sample:
            return sample["text"], sample["label"]
        elif "sentence" in sample:
            return sample["sentence"], sample["label"]
        elif "input_text" in sample:
            return sample["input_text"], sample["label"]
        else:
            raise ValueError("Unknown dict format:", sample)

    if isinstance(sample, (tuple, list)):
        return sample[0], sample[1]

    raise ValueError("Unknown sample format:", sample)

# Evaluation
def evaluate_dataset(model, dataset, dataset_name):
    correct = 0
    total = 0
    layers_used = []
    total_tokens = 0

    start = time.time()

    for sample in dataset:
        text, label = extract_text_label(sample)

        pred, layer, conf = model.classify(text, dataset_name)

        correct += (pred == label)
        total += 1
        layers_used.append(layer)

        total_tokens += len(model.tokenizer(text)["input_ids"])

    end = time.time()
    latency = (end - start) / total

    return {
        "metric": "accuracy",
        "score": correct / total,
        "avg_latency_sec": latency,
        "tokens_per_sec": total_tokens / (end - start),
        "avg_layers_used": float(np.mean(layers_used)),
        "num_samples": total
    }

In [9]:
results_table = []

print("Running BASELINE DISTILBERT finetuned on SST2")

baseline_model = DistilBERTBaselineClassifier()

# Print number of layers (for logging/reporting)
print("DistilBERT sst2 finetuned model layers:", baseline_model.num_layers)

for name, meta in cached_datasets.items():
    dataset = meta["data"]
    print(f"\nTesting BASELINE on {name}...")

    result = evaluate_dataset(baseline_model, dataset, name)
    print(name, result)

    results_table.append({
        "dataset": name,
        "threshold": None,
        "mode": "baseline",
        "model": "Distil-BERT",
        "metric": result["metric"],
        "score": float(result["score"]),
        "avg_latency_sec": float(result["avg_latency_sec"]),
        "tokens_per_sec": float(result["tokens_per_sec"]),
        "avg_layers_used": float(result["avg_layers_used"]),
        "num_samples": int(result["num_samples"]),
    })

Running BASELINE DISTILBERT finetuned on SST2
DistilBERT sst2 finetuned model layers: 6

Testing BASELINE on sst2...
sst2 {'metric': 'accuracy', 'score': 0.9105504587155964, 'avg_latency_sec': 0.011703385125606432, 'tokens_per_sec': 2150.1463513005733, 'avg_layers_used': 6.0, 'num_samples': 872}


In [12]:
results_table = []

print("Running BASELINE DISTILBERT uncased")

baseline_model_distil_uncased = DistilBERTBaselineClassifier(model_name="distilbert/distilbert-base-uncased")

# Print number of layers (for logging/reporting)
print("DistilBERT uncased model layers:", baseline_model_distil_uncased.num_layers)

for name, meta in cached_datasets.items():
    dataset = meta["data"]
    print(f"\nTesting BASELINE on {name}...")

    result = evaluate_dataset(baseline_model, dataset, name)
    print(name, result)

    results_table.append({
        "dataset": name,
        "threshold": None,
        "mode": "baseline",
        "model": "Distil-BERT",
        "metric": result["metric"],
        "score": float(result["score"]),
        "avg_latency_sec": float(result["avg_latency_sec"]),
        "tokens_per_sec": float(result["tokens_per_sec"]),
        "avg_layers_used": float(result["avg_layers_used"]),
        "num_samples": int(result["num_samples"]),
    })

Running BASELINE DISTILBERT uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBERT uncased model layers: 6

Testing BASELINE on sst2...
sst2 {'metric': 'accuracy', 'score': 0.9105504587155964, 'avg_latency_sec': 0.0046356514506383775, 'tokens_per_sec': 5428.361276433484, 'avg_layers_used': 6.0, 'num_samples': 872}

Testing BASELINE on imdb...
imdb {'metric': 'accuracy', 'score': 0.89072, 'avg_latency_sec': 0.006524844512939453, 'tokens_per_sec': 47015.833004394335, 'avg_layers_used': 6.0, 'num_samples': 25000}


In [13]:
print("Running BASELINE ROBERTA")

baseline_model_roberta = RoBERTaLargeBaselineClassifier()

print("RoBERTa Large model layers:", baseline_model_roberta.num_layers)

for name, meta in cached_datasets.items():
    dataset = meta["data"]
    print(f"\nTesting BASELINE on {name}...")

    result = evaluate_dataset(baseline_model_roberta, dataset, name)
    print(name, result)

    results_table.append({
        "dataset": name,
        "threshold": None,
        "mode": "baseline",
        "model": "ROBERTA-Large",
        "metric": result["metric"],
        "score": float(result["score"]),
        "avg_latency_sec": float(result["avg_latency_sec"]),
        "tokens_per_sec": float(result["tokens_per_sec"]),
        "avg_layers_used": float(result["avg_layers_used"]),
        "num_samples": int(result["num_samples"]),
    })

Running BASELINE ROBERTA


tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

RoBERTa Large model layers: 24

Testing BASELINE on sst2...


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

sst2 {'metric': 'accuracy', 'score': 0.9243119266055045, 'avg_latency_sec': 0.017467002803032552, 'tokens_per_sec': 1489.0462093136548, 'avg_layers_used': 24.0, 'num_samples': 872}

Testing BASELINE on imdb...


Token indices sequence length is longer than the specified maximum sequence length for this model (907 > 512). Running this sequence through the model will result in indexing errors


imdb {'metric': 'accuracy', 'score': 0.9558, 'avg_latency_sec': 0.02125024040222168, 'tokens_per_sec': 13873.635046932326, 'avg_layers_used': 24.0, 'num_samples': 25000}


In [14]:
df=pd.DataFrame(results_table)
df_sorted = (
    df.groupby("dataset", group_keys=True)
      .apply(lambda g: g.sort_values("score", ascending=False))
      .reset_index(drop=True)
)
df_sorted

/tmp/ipython-input-3662340674.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sort_values("score", ascending=False))


,dataset,threshold,mode,model,metric,score,avg_latency_sec,tokens_per_sec,avg_layers_used,num_samples
0,imdb,None,baseline,ROBERTA-Large,accuracy,0.955800,0.021250,13873.635047,24.0,25000
1,imdb,None,baseline,Distil-BERT,accuracy,0.890720,0.006525,47015.833004,6.0,25000
2,sst2,None,baseline,ROBERTA-Large,accuracy,0.924312,0.017467,1489.046209,24.0,872
3,sst2,None,baseline,Distil-BERT,accuracy,0.910550,0.004636,5428.361276,6.0,872
